In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
data = pd.read_csv("homework_1.1.csv")
print(data.head())
X = data[['X1', 'X2', 'X3']]
y = data['Y']
model = LinearRegression()
model.fit(X, y)
print("Intercept:", model.intercept_)
print("Coefficients:", model.coef_)
y_pred = model.predict(X)
mse = mean_squared_error(y, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y, y_pred)
print("MSE:", mse)
print("RMSE:", rmse)
print("R^2:", r2)

   Unnamed: 0        X1        X2        X3          Y
0           0 -0.440646 -0.390227  0.156718  -0.877671
1           1 -3.810099 -1.304665 -1.105117 -10.130388
2           2 -1.425451 -0.340049  1.115908   0.284068
3           3 -1.325750  0.161906 -0.254670  -1.994344
4           4  3.120263  1.487343 -1.164839   2.030030
Intercept: 0.0026430033841688405
Coefficients: [1.00713766 1.96456859 2.97548854]
MSE: 0.2509162964680301
RMSE: 0.5009154584039408
R^2: 0.9907160704940021


In [14]:
#Simple Regression for Xi

slopes_alone = {}
for col in ['X1', 'X2', 'X3']:
    model_simple = LinearRegression()
    Xi = data[[col]]
    model_simple.fit(Xi,y)
    slopes_alone[col] = model_simple.coef_[0]

print(slopes_alone)

{'X1': np.float64(1.8417610991461835), 'X2': np.float64(4.083612579373998), 'X3': np.float64(3.097041202049842)}


In [15]:
#Multiple Regressions
slopes_multi = dict(zip(['X1','X2','X3'], model.coef_))

print(slopes_multi)

{'X1': np.float64(1.0071376550530298), 'X2': np.float64(1.964568594874394), 'X3': np.float64(2.9754885351634344)}


In [16]:
differences = {col: abs(slopes_alone[col] - slopes_multi[col]) for col in slopes_alone}
print("Differences:", differences)

Differences: {'X1': np.float64(0.8346234440931537), 'X2': np.float64(2.119043984499604), 'X3': np.float64(0.12155266688640776)}
